In [1]:
%matplotlib inline
import time
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
import torch.nn.functional as F

from sklearn import preprocessing

In [2]:
# !pip install http://download.pytorch.org/whl/cu90/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

In [3]:
# !pip install torchvision

In [6]:
torch.__version__

'0.3.0.post4'

In [3]:
np.random.seed(0)

In [4]:
INPUT_SIZE = 224
# NUM_CLASSES = 16
root_dir = '/home/paul/share/dog/'
labels = pd.read_csv(join(root_dir, 'labels.csv'))
sample_submission = pd.read_csv(join(root_dir, 'sample_submission.csv'))
print(len(listdir(join(root_dir, 'train'))), len(labels))
print(len(listdir(join(root_dir, 'test'))), len(sample_submission))

10222 10222
10357 10357


In [5]:
sample_submission.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,001a5f3114548acdefa3d4da05474c2e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


In [8]:
# selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(NUM_CLASSES).index)
# labels = labels[labels['breed'].isin(selected_breed_list)]
le = preprocessing.LabelEncoder()
labels['class'] = le.fit_transform(labels['breed'])
# labels['target'] = 1
# labels_pivot = labels.pivot('id', 'breed', 'target').reset_index().fillna(0)

train = labels.sample(frac=0.9)
valid = labels[~labels['id'].isin(train['id'])]
test = pd.DataFrame(sample_submission['id'])
print(len(train), len(valid), len(test))

9200 1022 10357


In [9]:
class DogsDataset(Dataset):
    def __init__(self, labels, root_dir, train = True, transform=None):
        self.train = train
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img_name = '{}.jpg'.format(self.labels.iloc[idx, 0])
        fullpath = join(self.root_dir, img_name)
        image = Image.open(fullpath)
        if self.transform:
            image = self.transform(image)
            
        if self.train:
            labels = self.labels.iloc[idx, 2]
            return [image, labels]
        else:
            ids = self.labels.iloc[idx, 0]
            return image, ids

In [12]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)

# ds_trans = transforms.Compose([transforms.Scale(224),
#                                transforms.CenterCrop(224),
#                                transforms.ToTensor(),
#                                normalize])

ds_trans = transforms.Compose([transforms.Scale(224),
                               transforms.CenterCrop(224),
                               transforms.RandomSizedCrop(224), 
                               transforms.RandomHorizontalFlip(),
                               transforms.ToTensor(),
                               normalize])

dv_trans = transforms.Compose([transforms.Scale(256),
                               transforms.CenterCrop(224),
                               transforms.ToTensor(),
                               normalize])

train_ds = DogsDataset(train, join(root_dir,'train'), train=True, transform=ds_trans)
valid_ds = DogsDataset(valid, join(root_dir,'train'), train=True, transform=dv_trans)
test_ds = DogsDataset(test, join(root_dir,'test'), train=False, transform=dv_trans)

train_dl = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=4)
valid_dl = DataLoader(valid_ds, batch_size=4, shuffle=True, num_workers=4)
test_dl = DataLoader(test_ds, batch_size=4, shuffle=False, num_workers=4)

In [13]:
def imshow(axis, inp):
    """Denormalize and show"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    axis.imshow(inp)

In [15]:
# img, label = next(iter(train_dl))
# print(img.size(), label.size())
# fig = plt.figure(1, figsize=(16, 4))
# grid = ImageGrid(fig, 111, nrows_ncols=(1, 4), axes_pad=0.05)    
# for i in range(img.size()[0]):
#     ax = grid[i]
#     imshow(ax, img[i])

In [18]:
use_gpu = torch.cuda.is_available()
resnet = models.resnet50(pretrained=True)
inputs, labels = next(iter(train_dl))
if use_gpu:
    resnet = resnet.cuda()
    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())   
else:
    inputs, labels = Variable(inputs), Variable(labels)
outputs = resnet(inputs)
outputs.size()

torch.Size([4, 1000])

In [17]:
def train_model(dataloders, model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    use_gpu = torch.cuda.is_available()
    best_model_wts = model.state_dict()
    best_acc = 0.0
    dataset_sizes = {'train': len(dataloders['train'].dataset), 
                     'valid': len(dataloders['valid'].dataset)}

    for epoch in range(num_epochs):
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloders[phase]:
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                optimizer.zero_grad()

                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
#                 print(outputs, labels)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                train_epoch_loss = running_loss / dataset_sizes[phase]
                train_epoch_acc = running_corrects / dataset_sizes[phase]
            else:
                valid_epoch_loss = running_loss / dataset_sizes[phase]
                valid_epoch_acc = running_corrects / dataset_sizes[phase]
                
            if phase == 'valid' and valid_epoch_acc > best_acc:
                best_acc = valid_epoch_acc
                best_model_wts = model.state_dict()

        print('Epoch [{}/{}] train loss: {:.4f} acc: {:.4f} ' 
              'valid loss: {:.4f} acc: {:.4f}'.format(
                epoch, num_epochs - 1,
                train_epoch_loss, train_epoch_acc, 
                valid_epoch_loss, valid_epoch_acc))
            
    print('Best val Acc: {:4f}'.format(best_acc))
    print('Time for 1 epoch: {:.4f})'.format(time.time()-since))
    model.load_state_dict(best_model_wts)
    return model

In [12]:
# resnet = models.resnet50(pretrained=True)
# freeze all model parameters
for param in resnet.parameters():
    param.requires_grad = False

# new final layer with 120 classes
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, 120)
# if use_gpu:
#     resnet = resnet.cuda()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet.fc.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

dloaders = {'train':train_dl, 'valid':valid_dl}

In [13]:
start_time = time.time()
model = train_model(dloaders, resnet, criterion, optimizer, exp_lr_scheduler, num_epochs=1)
print('Training time: {:10f} minutes'.format((time.time()-start_time)/60))

Epoch [0/24] train loss: 0.7640 acc: 0.3237 valid loss: 0.2474 acc: 0.7241
Epoch [1/24] train loss: 0.4092 acc: 0.5942 valid loss: 0.1972 acc: 0.7583
Epoch [2/24] train loss: 0.3269 acc: 0.6590 valid loss: 0.1613 acc: 0.8170
Epoch [3/24] train loss: 0.2918 acc: 0.6822 valid loss: 0.1588 acc: 0.7965
Epoch [4/24] train loss: 0.2666 acc: 0.7078 valid loss: 0.1542 acc: 0.8268
Epoch [5/24] train loss: 0.2529 acc: 0.7177 valid loss: 0.1643 acc: 0.8023
Epoch [6/24] train loss: 0.2402 acc: 0.7246 valid loss: 0.1653 acc: 0.7965
Epoch [7/24] train loss: 0.2087 acc: 0.7710 valid loss: 0.1586 acc: 0.8141
Epoch [8/24] train loss: 0.2038 acc: 0.7798 valid loss: 0.1504 acc: 0.8209
Epoch [9/24] train loss: 0.2069 acc: 0.7751 valid loss: 0.1559 acc: 0.8190
Epoch [10/24] train loss: 0.2068 acc: 0.7700 valid loss: 0.1516 acc: 0.8170
Epoch [11/24] train loss: 0.2022 acc: 0.7813 valid loss: 0.1581 acc: 0.8112
Epoch [12/24] train loss: 0.2024 acc: 0.7812 valid loss: 0.1560 acc: 0.8121
Epoch [13/24] train lo

Index(['id', 'affenpinscher', 'afghan_hound', 'african_hunting_dog',
       'airedale', 'american_staffordshire_terrier', 'appenzeller',
       'australian_terrier', 'basenji', 'basset',
       ...
       'toy_poodle', 'toy_terrier', 'vizsla', 'walker_hound', 'weimaraner',
       'welsh_springer_spaniel', 'west_highland_white_terrier', 'whippet',
       'wire-haired_fox_terrier', 'yorkshire_terrier'],
      dtype='object', length=121)

In [24]:
if use_gpu:
    res_list = torch.FloatTensor().cuda()
else:
    res_list = torch.FloatTensor()

ids_list = []
for i, data in enumerate(test_dl):
    images, ids = data
#     images = Variable(images)
    if use_gpu:
        images = Variable(images.cuda())
    else:
        images = Variable(images)
    
    outputs = model(images)
    
    outputs_softmax = F.softmax(outputs)
    res_list = torch.cat((res_list, outputs_softmax.data),0)
    ids_list.append(ids)


In [25]:
submission_file = pd.DataFrame(list(res_list))
submission_file = pd.concat([sample_submission['id'],submission_file], axis=1)
submission_file.columns = list(sample_submission.columns)

In [26]:
submission_file.to_csv('/home/paul/share/dog/submission_file_resnet.csv', index=False)